In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/melting-point/train.csv')
test_df = pd.read_csv('/kaggle/input/melting-point/test.csv')

In [3]:
train_df.head()

,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1704,CC#CC(=O)O,351.15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2526,CCCCC(S)C,126.15,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_df.isnull().sum()[train_df.isnull().sum() >= 1] #no null values

Series([], dtype: int64)

In [5]:
train_df['SMILES'].nunique() #all unique values

2662

# MODELLING

In [6]:
X = train_df.drop(['id','SMILES','Tm'],axis=1)
y = train_df['Tm']

In [7]:
X.describe()

,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,Group 9,Group 10,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
count,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,2662.000000,...,2662.000000,2662.000000,2662.0,2662.000000,2662.0,2662.0,2662.0,2662.0,2662.0,2662.0
mean,1.020661,1.582269,0.114951,0.036439,0.036814,0.022915,0.013524,0.010143,0.007137,0.000751,...,0.001127,0.000751,0.0,0.001878,0.0,0.0,0.0,0.0,0.0,0.0
std,1.353640,3.738416,0.425964,0.208307,0.205516,0.164037,0.118732,0.117480,0.092695,0.027405,...,0.033558,0.027405,0.0,0.058126,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,16.000000,42.000000,6.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,...,1.000000,1.000000,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#drop empty features
def drop_features(df):
    for col in df:
        if df[col].sum() == 0:
            df = df.drop(col,axis=1)
    return df

In [9]:
#X = drop_features(X)

In [10]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
#random forest
model = RandomForestRegressor()
model.fit(X_train,y_train)
rf_pred = model.predict(X_val)

mean_absolute_error(y_val,rf_pred)

36.74957542347892

In [12]:
#decisiontree
model = DecisionTreeRegressor()
model.fit(X_train,y_train)
dt_pred = model.predict(X_val)

mean_absolute_error(y_val,dt_pred)

44.589199499687304

In [13]:
#XGB
model = XGBRegressor()
model.fit(X_train,y_train)
xgb_pred = model.predict(X_val)

mean_absolute_error(y_val,xgb_pred)

36.11422207887803

In [14]:
#LGBM
model = LGBMRegressor()
model.fit(X_train,y_train)
lgbm_pred = model.predict(X_val)

#MAE
mean_absolute_error(y_val,lgbm_pred)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 349
[LightGBM] [Info] Number of data points in the train set: 2129, number of used features: 80
[LightGBM] [Info] Start training from score 277.791617


42.85390541581454

Since my XGB model performed the best, I will work with that. Let me try fine tuning the parameters and hopefully improve performance of the model

# MODEL IMPROVEMENT

In [15]:
#XGB
model = XGBRegressor(
    booster = 'gbtree', 
    learning_rate = 0.4, 
    max_depth = None, #increasing can lead to overfitting, deafult = None
    subsample = 1, #range 0-1, deafult = NONE
    tree_method = 'hist',
    max_bin = None
    
)
model.fit(X_train,y_train)
xgb_pred = model.predict(X_val)

mean_absolute_error(y_val,xgb_pred)/y.mean()

0.1274851611552441

In [16]:
def custom_mae(y_true, y_pred):
    mae = np.sqrt(mean_absolute_error(y_true, y_pred))
    return mae
    
#param_grid = {
#     'booster' : ['gbtree',None], 
#    'learning_rate' : [None,0.5,1], 
#   'max_depth' : [None,2.5,6], #increasing can lead to overfitting, deafult = None
#    'subsample' : [None,0.5,1], #range 0-1, deafult = NONE
#   'tree_method' : [None,'hist'], #default = none
#    'max_bin' : [None,256] #default = none
#}

#define scoring metric
#scorer = make_scorer(custom_mae(y_val, xgb_pred),greater_is_better=False)

#set up and run gridsearchcv
#grid_search = GridSearchCV(estimator=model,
#                           param_grid=param_grid,
#                           scoring='neg_mean_absolute_error',
#                           cv=5,
#                           n_jobs=-1)  # use all cores

#grid_search.fit(X_train, y_train)

#check result
#print("Best Parameters:", grid_search.best_params_)
#print("Best Score:", grid_search.best_score_)

In [17]:
#model.get_params()

# SUBMISSION

In [18]:
X_test = test_df.drop(['id','SMILES'],axis=1)

In [19]:
predictions = model.predict(X_test)

In [20]:
submission = test_df[['id']]

In [21]:
submission['Tm'] = predictions

/tmp/ipykernel_13/2654297288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Tm'] = predictions


In [22]:
submission = submission.reset_index(drop=True)

In [23]:
submission.to_csv('submission.csv', index=False)

# RESOURCES
1. https://www.rdkit.org/docs/GettingStartedInPython.html
2. https://github.com/rdkit/rdkit
3. https://xgboost.readthedocs.io/en/stable/parameter.html